In [46]:
%load_ext autoreload
%autoreload 2
%aimport -jax
%aimport -jaxlib

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [47]:
import stanza.envs as envs
import stanza.policies as policies

import jax.flatten_util
import jax
import jax.numpy as jnp

from jax.random import PRNGKey
from stanza.util.random import PRNGSequence
from stanza.util.logging import logger

rng = PRNGSequence(42)
env = envs.create("quadrotor")

In [48]:
print(env.reset(next(rng)))

State(x=Array(-0.9451697, dtype=float32), z=Array(0.86506915, dtype=float32), phi=Array(0., dtype=float32), x_dot=Array(0., dtype=float32), z_dot=Array(0., dtype=float32), phi_dot=Array(0., dtype=float32))


In [49]:
# first step: generate expert trajectories 
from stanza.policies.mpc import MPC
from stanza.solver.ilqr import iLQRSolver
my_horizon = 100
solver_t = iLQRSolver()
expert_policy=MPC(
            # Sample action
            action_sample=env.sample_action(PRNGKey(0)),
            cost_fn=env.cost, 
            model_fn=env.step,
            horizon_length=100,
            solver=solver_t,
            receed=False
        )

def rollout_policy(rng_key, my_pol):
    # random init angle and angular velocity
    x_0 = env.reset(rng_key) 
    roll = policies.rollout(model = env.step,
                     state0 = x_0,
                     policy = my_pol,
                     length = my_horizon,
                     last_state = False)
    
    return roll.states, roll.actions

def batch_roll(rng_key, num_t, my_pol):
    roll_fun = jax.vmap(rollout_policy,in_axes=(0,None))
    rng_keys = jax.random.split(rng_key,num_t)
    return roll_fun(rng_keys,my_pol)


In [50]:
from stanza.data import Data, PyTreeData
expert_data = batch_roll(PRNGKey(42), 200,expert_policy)
expert_data  = Data.from_pytree(expert_data)
expert_data = expert_data.map(lambda x: Data.from_pytree(x))
expert_data = PyTreeData.from_data(expert_data.flatten(), chunk_size=4096)

In [51]:
from stanza.reporting.wandb import WandbDatabase

db = WandbDatabase("dpfrommer-projects/quadrotor_mlp").create()
logger.info(f"Logging to [blue]{db.name}[/blue]")


[19:11:17] INFO   - Logging to mild-snowball-16                                                     ]8;id=951324;file:///tmp/ipykernel_8675/3449362941.py\3449362941.py]8;;\:]8;id=674049;file:///tmp/ipykernel_8675/3449362941.py#4\4]8;;\

In [52]:
from stanza.nets.mlp import MLP
import chex

action_flat, action_uf = jax.flatten_util.ravel_pytree(env.sample_action(PRNGKey(0)))
state_flat, state_uf = jax.flatten_util.ravel_pytree(env.sample_state(PRNGKey(0)))
model = MLP([100, 100, 100, action_flat.shape[0]])

def loss_fn(state, params, _rng_key, sample):
    x, y = sample
    y_flat, _ = jax.flatten_util.ravel_pytree(y)
    x_flat, _ = jax.flatten_util.ravel_pytree(x)
    chex.assert_equal_shape([y_flat, action_flat])
    chex.assert_equal_shape([x_flat, state_flat])
    a_flat = model.apply(params, x_flat)
    loss = jnp.sum(jnp.square(y_flat-a_flat))
    stats = {'loss': loss}
    return state, loss, stats

def model_policy(params, input):
    x_flat, _ = jax.flatten_util.ravel_pytree(input.observation)
    a_flat = model.apply(params, x_flat)
    action = action_uf(a_flat)
    return policies.PolicyOutput(action)


In [53]:
from stanza.train import Trainer, batch_loss
from stanza.train.validate import Validator
from stanza.util.loop import every_kth_iteration, every_iteration, LoggerHook
from stanza.util.rich import ConsoleDisplay, StatisticsTable, LoopProgress
from stanza.reporting.jax import JaxDBScope

import stanza
import optax

iterations = 20000
optimizer = optax.adamw(optax.cosine_decay_schedule(1e-3, iterations), weight_decay=1e-4)

display = ConsoleDisplay()
display.add("train", StatisticsTable(), interval=100)
display.add("train", LoopProgress(), interval=100)

def val_fn(_, params, _rng_key, rng_key):
    x0 = env.reset(rng_key)
    policy = stanza.Partial(model_policy, params)
    rollout = policies.rollout(env.step, x0,
                               policy, length=100, 
                               last_state=False)
    cost = env.cost(rollout.states, rollout.actions)
    stats = {'cost': cost}
    return stats

def batch_val_fn(_, params, _rng_key, rng_keys):
    bval_fn = jax.vmap(val_fn, in_axes=(None, None, None, 0))
    stats = bval_fn(None, params, _rng_key, rng_keys)
    return jax.tree_map(lambda x: jnp.mean(x), stats)

validator = Validator(next(rng), 
        Data.from_pytree(jax.random.split(next(rng), 64)),
        condition=every_kth_iteration(1),
        stat_fn=batch_val_fn)

dbs = JaxDBScope(db)

with display as display_handle, dbs as dbs_handle:
    logger_hook = LoggerHook(every_kth_iteration(1000))
    db_logger_hook = dbs_handle.statistic_logging_hook(log_cond=every_kth_iteration(1), buffer=100)
    trainer = Trainer(
        loss_fn=batch_loss(loss_fn), batch_size=128,
        optimizer=optimizer,
        max_iterations=iterations,
        train_hooks=[validator, db_logger_hook, logger_hook,
                     display_handle.train]
    )
    
    logger.info("Initializing model...")
    init_params = model.init(next(rng), state_flat)
    logger.info("Training...")
    params = trainer.train(expert_data, rng_key=next(rng), init_params=init_params, jit=True)


Output()

[19:11:18] INFO   - Initializing model...                                                          ]8;id=615020;file:///tmp/ipykernel_8675/3682103474.py\3682103474.py]8;;\:]8;id=631349;file:///tmp/ipykernel_8675/3682103474.py#50\50]8;;\

           INFO   - Training...                                                                    ]8;id=443566;file:///tmp/ipykernel_8675/3682103474.py\3682103474.py]8;;\:]8;id=37875;file:///tmp/ipykernel_8675/3682103474.py#52\52]8;;\

           TRACE  - <Tracing> Tracing batch loss                                                    ]8;id=540541;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/train/__init__.py\__init__.py]8;;\:]8;id=753467;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/train/__init__.py#279\279]8;;\

           TRACE  - <Tracing> Tracing training                                                      ]8;id=152848;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/train/__init__.py\__init__.py]8;;\:]8;id=837558;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/train/__init__.py#122\122]8;;\

...

[19:11:55] INFO   - Iteration 16000:                                                                     ]8;id=367026;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py\loop.py]8;;\:]8;id=58789;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py#85\85]8;;\
           test.cost: 0.3064458966255188                                                                           
           train.loss: 0.0004190051113255322                                                                       

[19:11:57] INFO   - Iteration 17000:                                                                     ]8;id=564211;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py\loop.py]8;;\:]8;id=671708;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py#85\85]8;;\
           test.cost: 0.30652087926864624                                                                          
           train.loss: 0.0003684178227558732                                                                       

[19:11:59] INFO   - Iteration 18000:                                                                     ]8;id=816042;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py\loop.py]8;;\:]8;id=163782;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py#85\85]8;;\
           test.cost: 0.3082115352153778                                                                           
           train.loss: 0.0006959806778468192                                                                       

[19:12:01] INFO   - Iteration 19000:                                                                     ]8;id=573812;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py\loop.py]8;;\:]8;id=792579;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py#85\85]8;;\
           test.cost: 0.30720508098602295                                                                          
           train.loss: 0.0005080956034362316                                                                       

[19:12:03] INFO   - Iteration 20000:                                                                     ]8;id=271302;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py\loop.py]8;;\:]8;id=192721;file:///home/daniel/Documents/code/stable_imitation/stanza/stanza/util/loop.py#85\85]8;;\
           test.cost: 0.3063797652721405                                                                           
           train.loss: 0.00044235275709070265                                                                      